In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import csv
import cv2

%matplotlib inline

In [0]:
# base directory where you store your *.zip from lisa-traffic-light-dataset.zip
base_dir = 'drive/My Drive/traffic_lights_research/kaggle/'

# labels
labels_dir = 'Annotations/dayTrain/dayClip3/'

# image folder
image_dir = 'dayTrain/dayClip3/frames/'

In [0]:
# firstly, you need unzip Annotations.zip and dayTrain.zip
# !unzip /content/drive/My\ Drive/traffic_lights_research/kaggle/dayTrain.zip -d /content/drive/My\ Drive/traffic_lights_research/kaggle/
# !unzip /content/drive/My\ Drive/traffic_lights_research/kaggle/Annotations.zip -d /content/drive/My\ Drive/traffic_lights_research/kaggle/

In [41]:
annotation_name = 'frameAnnotationsBOX.csv'
traffic_df = pd.read_csv(base_dir + labels_dir + annotation_name, delimiter=';')
traffic_df.head()

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number
0,dayTraining/dayClip3--00001.jpg,warning,622,361,637,379,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,1,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,1
1,dayTraining/dayClip3--00002.jpg,warning,622,358,637,380,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,2,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,2
2,dayTraining/dayClip3--00003.jpg,warning,506,382,518,404,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,3,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,3
3,dayTraining/dayClip3--00004.jpg,warning,620,358,638,380,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4
4,dayTraining/dayClip3--00004.jpg,warning,504,382,519,404,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4


In [42]:
traffic_df["Upper left corner X"] = traffic_df["Upper left corner X"].apply(str)
traffic_df["Upper left corner Y"] = traffic_df["Upper left corner Y"].apply(str)
traffic_df["Lower right corner X"] = traffic_df["Lower right corner X"].apply(str)
traffic_df["Lower right corner Y"] = traffic_df["Lower right corner Y"].apply(str)
traffic_df["Lower right corner Y"].describe()

count     1131
unique     226
top        304
freq       175
Name: Lower right corner Y, dtype: object

In [43]:
print('Number of images: ', traffic_df.Filename.size)
print('Number of unique images: ', traffic_df.Filename.nunique())

Number of images:  1131
Number of unique images:  547


In [44]:
print('Number of unique annotation tags: ', traffic_df['Annotation tag'].nunique())

Number of unique annotation tags:  3


In [45]:
traffic_df.Filename[traffic_df.Filename == "dayTraining/dayClip3--00004.jpg"]

3    dayTraining/dayClip3--00004.jpg
4    dayTraining/dayClip3--00004.jpg
Name: Filename, dtype: object

In [46]:
same_filenames = traffic_df.index[traffic_df['Filename'] == 'dayTraining/dayClip3--00004.jpg'].tolist()
same_filenames

[3, 4]

In [47]:
print('Number of images with red light: ', traffic_df.Filename[traffic_df['Annotation tag'] == 'stop'].size)
print('Number of images with yellow light: ', traffic_df.Filename[traffic_df['Annotation tag'] == 'warning'].size)
print('Number of images with green light: ', traffic_df.Filename[traffic_df['Annotation tag'] == 'go'].size)

Number of images with red light:  953
Number of images with yellow light:  60
Number of images with green light:  118


Generate unique names for each image. Let's create new column



In [48]:
traffic_df["image_name"] = traffic_df.index
traffic_df["image_name"] = traffic_df["image_name"].apply(str)
traffic_df["image_name"] = 'img' + traffic_df["image_name"] + '.jpg'
traffic_df.head()

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number,image_name
0,dayTraining/dayClip3--00001.jpg,warning,622,361,637,379,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,1,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,1,img0.jpg
1,dayTraining/dayClip3--00002.jpg,warning,622,358,637,380,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,2,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,2,img1.jpg
2,dayTraining/dayClip3--00003.jpg,warning,506,382,518,404,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,3,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,3,img2.jpg
3,dayTraining/dayClip3--00004.jpg,warning,620,358,638,380,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4,img3.jpg
4,dayTraining/dayClip3--00004.jpg,warning,504,382,519,404,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4,img4.jpg


In [49]:
labels_df = traffic_df[['image_name', 'Annotation tag', 'Upper left corner X', 'Upper left corner Y', 'Lower right corner X', 'Lower right corner Y', 'Filename']].copy()
labels_df.columns = ['file_name', 'class', 'xmin', 'ymin', 'xmax', 'ymax', 'image_name']
labels_df.head()

,file_name,class,xmin,ymin,xmax,ymax,image_name
0,img0.jpg,warning,622,361,637,379,dayTraining/dayClip3--00001.jpg
1,img1.jpg,warning,622,358,637,380,dayTraining/dayClip3--00002.jpg
2,img2.jpg,warning,506,382,518,404,dayTraining/dayClip3--00003.jpg
3,img3.jpg,warning,620,358,638,380,dayTraining/dayClip3--00004.jpg
4,img4.jpg,warning,504,382,519,404,dayTraining/dayClip3--00004.jpg


In [0]:
for index in range(labels_df.shape[0]):
    name = base_dir + image_dir + labels_df.iloc[0]['image_name'].split('/')[-1]
    image = cv2.imread(name) # using the orig name, read the image
    
    labels_df['height'] = image.shape[0]
    labels_df['width'] = image.shape[1]
    labels_df['height'] = labels_df['height'].apply(str)
    labels_df['width'] = labels_df['width'].apply(str)
    
    labels_df['image_path'] = name
    labels_df['image_path'] = labels_df['image_path'].apply(str)

In [51]:
labels_df.head()

,file_name,class,xmin,ymin,xmax,ymax,image_name,height,width,image_path
0,img0.jpg,warning,622,361,637,379,dayTraining/dayClip3--00001.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
1,img1.jpg,warning,622,358,637,380,dayTraining/dayClip3--00002.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
2,img2.jpg,warning,506,382,518,404,dayTraining/dayClip3--00003.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
3,img3.jpg,warning,620,358,638,380,dayTraining/dayClip3--00004.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
4,img4.jpg,warning,504,382,519,404,dayTraining/dayClip3--00004.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [0]:
rand_state = 100
y_train_df, y_test_df = train_test_split(labels_df, test_size=0.2, random_state=rand_state)

In [0]:
y_train_df = y_train_df.copy()
y_test_df = y_test_df.copy()

In [55]:
print('y_train_df shape: ', y_train_df.shape)
print('y_test_df shape: ', y_test_df.shape)

y_train_df shape:  (904, 10)
y_test_df shape:  (227, 10)


In [56]:
y_train_df.head()

,file_name,class,xmin,ymin,xmax,ymax,image_name,height,width,image_path
580,img580.jpg,stop,619,77,661,140,dayTraining/dayClip3--00329.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
515,img515.jpg,stop,893,256,932,328,dayTraining/dayClip3--00307.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
46,img46.jpg,warning,780,356,792,374,dayTraining/dayClip3--00060.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
424,img424.jpg,stop,628,202,661,251,dayTraining/dayClip3--00277.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
826,img826.jpg,stop,960,205,1020,304,dayTraining/dayClip3--00443.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...


In [57]:
y_test_df.head()

,file_name,class,xmin,ymin,xmax,ymax,image_name,height,width,image_path
1074,img1074.jpg,go,975,210,1023,290,dayTraining/dayClip3--00568.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
816,img816.jpg,stop,960,205,1020,304,dayTraining/dayClip3--00438.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
954,img954.jpg,stop,960,205,1020,304,dayTraining/dayClip3--00507.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
241,img241.jpg,stop,888,313,903,344,dayTraining/dayClip3--00212.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...
239,img239.jpg,stop,779,266,794,284,dayTraining/dayClip3--00212.jpg,960,1280,drive/My Drive/traffic_lights_research/kaggle/...


In [58]:
# Firstly create file to store train_labels.csv and test_labels.csv
from datetime import datetime
 
now = datetime.utcnow()
current_time = "{}-{}-{}--{}-{}".format(now.day, now.month, now.year, now.hour, now.minute)

# create train_labels.csv
train_labels_location = base_dir + labels_dir + 'train_labels-{}.csv'.format(current_time)
file = open(train_labels_location, "w") 
file.close() 

# create test_labels.csv
test_labels_location = base_dir + labels_dir + 'test_labels-{}.csv'.format(current_time)
file = open(test_labels_location, "w") 
file.close()

os.listdir(base_dir + labels_dir)

['frameAnnotationsBOX.csv',
 'frameAnnotationsBULB.csv',
 'train_labels-22-5-2019--17-38.csv',
 'test_labels-22-5-2019--17-38.csv']

In [0]:
y_train_df.to_csv(train_labels_location, index = False, header=True)
y_test_df.to_csv(test_labels_location, index = False, header=True)